# <span style="color:DarkGreen"> *Event Total Precipitation Comparison between Observations & The High Resolution Ensemble Forecast (HREF)* </span>

---

<div class="alert alert-block alert-success">
<b>Notes:</b> Review ASOS Calculations
</div>

#### **HREF Members Used:**

###### *- Advanced Research Weather (HRW WRF-ARW)*
###### *- Finite Volume Cubed Sphere (HRW WRF-FV3)*
###### *- National Severe Storms Laboratory (HRW WRF-NSSL)*
###### *- North American Model (NAM 3km CONUS)*
###### *- High Resolution Rapid Refresh (HRRR)*

#### **Observations Used:**
###### *- New York State Mesonet (NYSM)*
###### *- Automated Surface Observing Systems (ASOS)*

### Inputs

In [1]:
# HREF Initialization Time & NYSM Observation Starting Time
year = 2022 
month = 2 
day = 17 
hour = 0 #12
minute = 0 

# HREF Initialization Hours
starthour = 36 #0,3,6,9,12,15,18
endhour = 36 #2,5,8,11,14,17,20
inc = 1

### Imports

In [2]:
%matplotlib inline

# core
import os
import sys
import glob
import math
import cfgrib
import requests
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib as mpl

# Collections
from collections import Counter
from functools import reduce

# netCDF4
import netCDF4 as nc
from netCDF4 import Dataset

# datetime
import datetime as dt
from datetime import datetime,timedelta

#cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeat 
from cartopy import feature as cfeature

# matplotlib
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patheffects as PathEffects
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.dates import DateFormatter, AutoDateLocator,HourLocator,DayLocator,MonthLocator

# metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import StationPlot, USCOUNTIES
from metpy.calc import wind_speed, wind_direction, relative_humidity_from_dewpoint, wind_components

# Custom 
import cmap_atmos as cmatmos #custom colormaps
cmap_apcp, norm_apcp, apcp_levs = cmatmos.precip_nws()

### File Grabbing 

In [3]:
#Load in field sites
sites = pd.read_csv('/network/rit/home/je845911/minlab/je845911/data/nysm/site_locations.txt')

#Getting coordinates for mesonet data because 2022 files dont have them
fpath_coords = '/network/rit/lab/minderlab_rit/NYSM/standard/netCDF/2019/20190902.nc'

# ASOS File Grabbing 
df_NY = pd.read_csv('/network/rit/home/je845911/minlab/je845911/data/asos/five_pannel/IOP4_NY_ASOS.csv')
df_VT = pd.read_csv('/network/rit/home/je845911/minlab/je845911/data/asos/five_pannel/IOP4_VT_ASOS.csv')
df_QE = pd.read_csv('/network/rit/home/je845911/minlab/je845911/data/asos/five_pannel/IOP4_QE_ASOS_PRECIP.csv')
df_ON = pd.read_csv('/network/rit/home/je845911/minlab/je845911/data/asos/five_pannel/IOP4_ON_ASOS.csv')

# NYSM File Grabbing
hour_delta = 0
minute_delta = 60
year_str = str(year)
month_str = str(month).zfill(2)
day_str = str(day).zfill(2)
hour_str = str(hour).zfill(2)
day2_str = str(day+1)
base_dir = '/network/rit/lab/minderlab_rit/NYSM'
standard_dir = '/standard/netCDF/2022'
standard_1 = base_dir + standard_dir + '/' + year_str + month_str + day_str + '.nc'
standard_2 = base_dir + standard_dir + '/' + year_str + month_str + day2_str + '.nc'
print('Gathering NYSM files...')
print(standard_1)
print(standard_2)
print()
NYSM_temp = xr.open_mfdataset([standard_1,standard_2])
ds_coords = xr.open_dataset(fpath_coords)

####################################################################################
# HREF File Grabbing

href_base_dir = '/network/rit/lab/minderlab_rit/je845911/data/href'
yyyymmddhh_str = year_str + month_str + day_str + hour_str
yyyymmdd_str = yyyymmddhh_str[0:8]
yyyy_str = yyyymmddhh_str[0:4]
endhour = endhour +1
fhrs = np.arange(starthour,endhour,inc)
fhrs_adj = np.arange(0,starthour,1)
fhrs_adj_nam = np.arange(0,starthour+1,1)
length = len(fhrs)

arw, fv3, nssl, ncep, nam = [],[],[],[],[]
for fhr in fhrs:
    arw.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/hiresw_conusarw_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')
    fv3.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/hiresw_conusfv3_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')
    nssl.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/hiresw_conusnssl_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')
    ncep.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/hrrr_ncep_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')
    nam.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/nam_conusnest_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')

fv3adj = []
for fhr in fhrs_adj:
    fv3adj.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/hiresw_conusfv3_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')

namadj = []
for fhr in fhrs_adj_nam:
    namadj.append(f'{href_base_dir}/{yyyy_str}/{yyyymmdd_str}/nam_conusnest_' + f'{yyyymmddhh_str}f0'+str(fhr).zfill(2)+'.grib2')
    
print('Gathering files from the ARW...')
for fpath in arw:
    print(fpath)
print()
print('Gathering files from the FV3...')
for fpath in fv3adj:
    print(fpath)
for fpath in fv3:
    print(fpath)
print()
print('Gathering files from the NSSL...')
for fpath in nssl:
    print(fpath)
print()
print('Gathering files from the NCEP...')
for fpath in ncep:
    print(fpath)
print()
print('Gathering files from the NAM...')
for fpath in namadj:
    print(fpath)
print()
for fpath in nam:
    print(fpath)
print()

Gathering NYSM files...
/network/rit/lab/minderlab_rit/NYSM/standard/netCDF/2022/20220217.nc
/network/rit/lab/minderlab_rit/NYSM/standard/netCDF/2022/20220218.nc

Gathering files from the ARW...
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusarw_2022021700f036.grib2

Gathering files from the FV3...
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f000.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f001.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f002.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f003.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f004.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f005.grib2
/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/h

### Observations

In [4]:
# WINTRE-MIX REGION
latN = 46.5
latS = 44
lonW = -77.0
lonE = -72.0
cLat = (latN + latS)/2
cLon = (lonW + lonE )/2

# Making both files station coordinates formating identical
ds_coords['station']=np.array([str(stn.values,'utf-8') for stn in ds_coords['station']])

# Adding Lat and Lon from 2019 NYSM file to the 2022 Files 
NYSM_temp = NYSM_temp.assign(lat = ds_coords['lat'])
NYSM_temp = NYSM_temp.assign(lon = ds_coords['lon'])

# Converting NYSM .nc files to Pandas Dataframes
NYSM_temp = NYSM_temp.to_dataframe()
NYSM_temp = NYSM_temp.reset_index(level=1)

# Cropping the dataset to only get data from the WINTRE-MIX region
NYSM_temp_cropped = NYSM_temp[
    (NYSM_temp["lat"] <= latN) & 
    (NYSM_temp["lat"] >= latS) & 
    (NYSM_temp["lon"] >= lonW) & 
    (NYSM_temp["lon"] <= lonE)
]

# Storing cropped NYSM Temperature dataset as "NYSM_temp_cropped"
NYSM_temp_cropped = NYSM_temp_cropped[NYSM_temp_cropped['temp_2m'] != 'M']

# Making a time to move through NYSM Data
valid_time = datetime(year,month,day,hour,minute)
valid_time_str = valid_time.strftime("%Y-%m-%d %H:%M") 
hr = valid_time+dt.timedelta(minutes=minute_delta)
time = hr.strftime("%Y-%m-%d %H:%M") 

dfs = [df_NY, df_ON, df_QE, df_VT] #Combining the ASOS dataframes from selected stations in Ontario, Quebec, and NY
df_merged = pd.concat(dfs)
df_merged_cropped = df_merged[
    (df_merged["lat"] <= latN) & 
    (df_merged["lat"] >= latS) & 
    (df_merged["lon"] >= lonW) & 
    (df_merged["lon"] <= lonE)
]

### 5 Pannel Output

In [5]:
frontend_fv3,frontend_nam,backend_fv3,backend_nam = np.zeros((1059,1799), dtype = 'float'),np.zeros((1059,1799), dtype = 'float'),np.zeros((1059,1799), dtype = 'float'),np.zeros((1059,1799), dtype = 'float')
fhrs_adj_nam = np.arange(0,starthour+1,3)
#fhrs_adj_nam_inc = np.arange(starthour+1,starthour+3,1)

# FV3 Fontend Precipitation Totals
for i in fhrs_adj:
    
    with xr.open_mfdataset(fv3adj[i],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as fv3_fc:
        fv3_apcp = fv3_fc['tp'].isel(valid_time=0)
        fv3_apcp = fv3_apcp/25.4
        fv3_apcp = np.float32(fv3_apcp)     
        frontend_fv3=np.add(fv3_apcp,frontend_fv3)
    
# NAM Fontend Precipitation Totals
for i in fhrs_adj_nam:
    print(i)
    with xr.open_mfdataset(namadj[i],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as nam_fc:
        nam_apcp = nam_fc['tp'].isel(valid_time=0)
        nam_apcp = nam_apcp/25.4
        nam_apcp = np.float32(nam_apcp)     
        frontend_nam = nam_apcp + frontend_nam

print(frontend_nam)
        
        
for i in fhrs:
    # Time 
    print("The passing of i is ", i)
    D = int(((i-starthour)/inc)) # Used for HREF File Reference  
    time = (hr) + timedelta(hours=float(i)) 
    adjtime = time - dt.timedelta(minutes=minute_delta)
    adjtimestr = adjtime.strftime("%Y-%m-%d %H:%M") 
    nysm_adjtime = time.strftime("%H:%M")
    time_delta = (hr) - timedelta(hours=hour_delta, minutes=minute_delta)
    time_delta_str = time_delta.strftime("%Y-%m-%d %H:%M") 
    titlestrend = adjtime.strftime("%Y-%m-%d %H:%M")
    
    # NYSM
    current_precip = (NYSM_temp_cropped['time'] == adjtime.strftime('%Y-%m-%d %H:%M'))
    current_p = NYSM_temp_cropped.loc[current_precip]
    current_p['precip_local'] = current_p['precip_local'].fillna(0)
    lats_NYSM_precip = current_p['lat']
    lons_NYSM_precip = current_p['lon']
    
    valid_precip = (NYSM_temp_cropped['time'] == valid_time.strftime('%Y-%m-%d %H:%M'))
    valid_p = NYSM_temp_cropped.loc[valid_precip]
    valid_p['precip_local'] = valid_p['precip_local'].fillna(0)
    
    eigthteenreset = (NYSM_temp_cropped['time'] == datetime(2022,2,18,0,0).strftime('%Y-%m-%d %H:%M'))
    eigthteen = NYSM_temp_cropped.loc[eigthteenreset]
    eigthteen['precip_local'] = eigthteen['precip_local'].fillna(0)
    
    if datetime.strptime('2022-02-17 12:00', "%Y-%m-%d %H:%M") > adjtime:# hr
        NYSM_precip = ((current_p['precip_local'])/25.4)
    elif datetime.strptime('2022-02-17 12:00', "%Y-%m-%d %H:%M") <= adjtime and datetime.strptime('2022-02-18 01:00', "%Y-%m-%d %H:%M") > adjtime:
        NYSM_precip = (((current_p['precip_local'] - valid_p['precip_local']))/25.4)
    else:
        NYSM_precip = ((((eigthteen['precip_local']-valid_p['precip_local']) + current_p['precip_local']))/25.4)
        
    # ASOS
    mask_ASOS = (df_merged_cropped['valid'] > time_delta_str) & (df_merged_cropped['valid'] <= str(adjtime))#adjtime
    ASOS_cropped = df_merged_cropped.loc[mask_ASOS]
    mask_ASOS_precip = (df_merged_cropped['valid'] > valid_time_str) & (df_merged_cropped['valid'] <= str(adjtime))
    ASOS_cropped_precip = df_merged_cropped.loc[mask_ASOS_precip]
    df_merged = ASOS_cropped.drop_duplicates(subset='station', keep='last', inplace=False) 
    cropped_ASOS_precip = ASOS_cropped_precip[ASOS_cropped_precip['p01i'] != 'M']
    cropped_ASOS_precip = cropped_ASOS_precip[cropped_ASOS_precip['p01i'] != 'T']
    cropped_ASOS_precip = cropped_ASOS_precip[cropped_ASOS_precip['p01i'] != '0.00']
    cropped_ASOS = cropped_ASOS_precip.drop(cropped_ASOS_precip.columns[[4,5,6,7,8,10,12,13,14]], axis=1)
    cropped_ASOS['valid'] = pd.to_datetime(cropped_ASOS['valid'])
    cropped_ASOS['Hour'] = pd.to_datetime(cropped_ASOS['valid'], infer_datetime_format=True).dt.hour
    cropped_ASOS['precip_max'] = cropped_ASOS.groupby(['station', 'Hour'])['p01i'].transform('max')
    cropped_ASOS.loc[cropped_ASOS['p01i'] == cropped_ASOS['precip_max']]
    cropped_ASOS['precip_max'] = cropped_ASOS['precip_max'].astype(float)
    cropped_ASOS = cropped_ASOS.sort_values('precip_max').drop_duplicates(subset=['Hour', 'station'], keep='last')
    cropped_ASOS = cropped_ASOS.groupby(["station", 'lat', 'lon']).precip_max.sum()
    ASOS_precip = cropped_ASOS.to_frame().reset_index()
    lats_ASOS_precip = ASOS_precip['lat']
    lons_ASOS_precip = ASOS_precip['lon']
    precip_ASOS = ASOS_precip['precip_max']
    
    # HREF
    
    # ARW Total Precipitation
    with xr.open_mfdataset(arw[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as arw_fc:
        arw_apcp = arw_fc['tp'].isel(valid_time=0)
        arw_apcp = arw_apcp/25.4
     
    # FV3 Total Precipitation
    with xr.open_mfdataset(fv3[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as fv3_fc:
        fv3_apcp = fv3_fc['tp'].isel(valid_time=0)
        fv3_apcp = fv3_apcp/25.4
        backend_fv3 = np.add(fv3_apcp,backend_fv3)
        fv3_apcp = frontend_fv3 + backend_fv3
        
    # NSSL Total Precipitation
    with xr.open_mfdataset(nssl[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as nssl_fc:
        nssl_apcp = nssl_fc['tp'].isel(valid_time=0)
        nssl_apcp = nssl_apcp/25.4
        
    # NCEP Total Precipitation
    with xr.open_mfdataset(ncep[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as ncep_fc:
        ncep_apcp = ncep_fc['tp'].isel(valid_time=0)
        ncep_apcp = ncep_apcp/25.4
            
    print(i)
    print(starthour)
    
    if i == starthour:
        print("in first loop", (nam[D]))
        frontend_total = frontend_nam
    
    if i == starthour + 1:
        print("in second loop", (nam[D]))
        with xr.open_mfdataset(nam[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as nam_fc:
                hour_baseline = nam_fc['tp'].isel(valid_time=0)
                hour_baseline = hour_baseline/25.4
                hour_baseline = np.float32(hour_baseline)
                frontend_total = hour_baseline + frontend_nam #frontend_total

    if i == starthour + 2:
        print("in thrid loop", (nam[D]))
        with xr.open_mfdataset(nam[D],engine = 'cfgrib',filter_by_keys={'stepType': 'accum','typeOfLevel': 'surface','paramId':228228},concat_dim = 'valid_time',combine = 'nested') as nam_fc:
                hour_secondary = nam_fc['tp'].isel(valid_time=0)
                hour_secondary = hour_secondary/25.4
                hour_secondary = np.float32(hour_secondary)     
                frontend_total = hour_secondary + frontend_nam

    #########################
    nam_apcp = frontend_total
    print("FINALFINALFINAL")
    print(nam_apcp)
    print("FINALFINALFINAL")
    #########################
    
    #Mean Precipitation
    precip_mean = ((arw_apcp + fv3_apcp + nssl_apcp + ncep_apcp + nam_apcp)/5)
    
    # Lat & Lon
    LAT = arw_apcp['latitude']#arw_totp
    LON = arw_apcp['longitude']#arw_totp
    
    # Defiining Figure Features
    def features(ax):
        ax.set_extent ([lonW,lonE,latS,latN])
        ax.add_feature (cfeature.LAND.with_scale(res), zorder=1)
        ax.add_feature (cfeature.OCEAN.with_scale(res), zorder=1)
        ax.add_feature (cfeature.LAKES.with_scale(res), zorder=1)
        ax.add_feature(cfeature.COASTLINE.with_scale(res), zorder = 3)
        ax.add_feature (cfeature.STATES.with_scale(res), zorder = 3)
        ax.add_feature(USCOUNTIES.with_scale(county_scale),zorder= 3, linewidth = county_lw)
        
    def legend(ax):
        c='white'
        ASOS = mlines.Line2D([], [], color=c, marker='s', ls='', label='ASOS', markeredgecolor=color)
        NYSM = mlines.Line2D([], [], color=c, marker='o', ls='', label='NYSM', markeredgecolor=color)
        WINTRE_MIX = mlines.Line2D([], [], color=c, marker='$X$', ls='', label='WINTRE-MIX', markeredgecolor=color)
        ax.legend(loc = 'upper left',handles=[NYSM, ASOS, WINTRE_MIX],frameon=True,fontsize=12,shadow = True,edgecolor = 'black')
   
    # Title
    tl0 = 'High Resolution Essemble Forecast'
    tl1 = 'Observations from The New York State Mesonet, and ASOS'
    tl2 = f'Initilized: {valid_time_str} UTC, Forecast Hour: [{i}], Valid: {titlestrend} UTC'

    title_line = (tl0+ '\n' + tl2 + '\n' + tl1)
    variable = ' Total Precipitation (in)'
    model1_t = 'WRF-ARW:'
    model2_t = 'WRF-FV3:'
    model3_t = 'NSSL:'
    model4_t = 'HRRR NCEP:'
    model5_t = 'NAM NEST:'
    model6_t = 'Enssemble Mean:'
    titletime = valid_time.strftime("%Y%m%d%H") 
    savefiguretitle = f'precip_{str(titletime)}_{i}'

    # Expanded WINTRE-MIX region
    latN = 46.5
    latS = 43.75
    lonW = -77.0
    lonE = -72.0
    cLat = (latN + latS)/2
    cLon = (lonW + lonE )/2
    
    # ASOS & NYSM Features
    color ='black'
    fontsize = 12
    county_lw, county_scale = 1.0, '5m'
    cbarna = 'coolwarm' #'jet'
    
    # Figure Features
    res = '50m'
    proj = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
    msize = 110
    
    # Surface Temperature Figure
    fig = plt.figure(figsize=(36,28),dpi=100) # 36,28
    fig.suptitle(title_line, fontsize = 24)
    plt.subplots_adjust(hspace = 0.09)
    plt.subplots_adjust(wspace = -0.5)

    ax1 = fig.add_subplot(3,2,1, projection=proj)
    ax1.set_title(model1_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax1)  
    ax1.pcolormesh(LON,LAT,arw_apcp,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax1.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites')    
    ASOS = ax1.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax1, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax1.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax1, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax1)
    
    ax2 = fig.add_subplot(3,2,2, projection=proj)
    ax2.set_title(model2_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax2)
    ax2.pcolormesh(LON,LAT,fv3_apcp,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax2.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites')
    ASOS = ax2.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax2, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax2.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax2, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax2)
    
    ax3 = fig.add_subplot(3,2,3, projection=proj)
    ax3.set_title(model3_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax3)
    ax3.pcolormesh(LON,LAT,nssl_apcp,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax3.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites') 
    ASOS = ax3.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax3, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax3.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax3, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax3)
    
    ax4 = fig.add_subplot(3,2,4, projection=proj)
    title = ax4.set_title(model4_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax4)  
    ax4.pcolormesh(LON,LAT,ncep_apcp,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax4.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites')   
    ASOS = ax4.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax4, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax4.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax4, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax4)
    
    ax5 = fig.add_subplot(3,2,5, projection=proj)
    ax5.set_title(model5_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax5)   
    ax5.pcolormesh(LON,LAT,nam_apcp,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax5.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites')  
    ASOS = ax5.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax5, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax5.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax5, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax5)
    
    ax6 = fig.add_subplot(3,2,6, projection=proj)
    ax6.set_title(model6_t+variable,fontsize=16,loc = 'left',fontweight = 'bold')
    features(ax6)   
    ax6.pcolormesh(LON,LAT,precip_mean,shading = 'auto', cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 3)
    ax6.scatter(sites['lon'], sites['lat'], s = msize, c = 'black', marker = "$X$" , transform = ccrs.PlateCarree(), zorder = 4, label= 'WINTRE-MIX Sites')  
    ASOS = ax6.scatter(lons_ASOS_precip, lats_ASOS_precip, s=msize, c = precip_ASOS, cmap = cmap_apcp, norm = norm_apcp, transform = ccrs.PlateCarree(), zorder = 989, marker='s', label='ASOS', edgecolor=color)
    stationplot_ASOS = StationPlot(ax6, lons_ASOS_precip, lats_ASOS_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    NYSM = ax6.scatter(lons_NYSM_precip, lats_NYSM_precip, s = msize, c = NYSM_precip, cmap = cmap_apcp, norm=norm_apcp,transform = ccrs.PlateCarree(), zorder = 998, label='NYSM', edgecolor=color)
    stationplot_NYSM = StationPlot(ax6, lons_NYSM_precip, lats_NYSM_precip, transform=ccrs.PlateCarree(),fontsize=18, fontweight='bold', zorder=999)
    legend(ax6)
    
    colorbar_axes = fig.add_axes([0.275, 0.1, .476, .0125])# Left Bottom Width Height
    cbar = plt.colorbar(NYSM, ticks=[.25,1,2,4,8,15], cax = colorbar_axes, orientation = 'horizontal')
    cbar.ax.set_xlabel("Precipitation (in)",fontsize=18)
    cbar.ax.tick_params(labelsize=14)
    
    fig.savefig(savefiguretitle)

Can't read index file '/network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/hiresw_conusfv3_2022021700f010.grib2.923a8.idx'
Traceback (most recent call last):
  File "/network/rit/lab/snowclus/anaconda3_2021/envs/may21/lib/python3.8/site-packages/cfgrib/messages.py", line 535, in from_indexpath_or_filestream
    self = cls.from_indexpath(indexpath)
  File "/network/rit/lab/snowclus/anaconda3_2021/envs/may21/lib/python3.8/site-packages/cfgrib/messages.py", line 417, in from_indexpath
    index = pickle.load(file)
EOFError: Ran out of input


0
3
6
9
12
15
18
21
24
27
30
33
36
[[0.05167323 0.03198819 0.03444882 ...        nan        nan        nan]
 [0.06151575 0.04675197 0.03690945 ...        nan        nan        nan]
 [0.06643701 0.06397638 0.0492126  ...        nan        nan        nan]
 ...
 [0.06397638 0.05905512 0.07135827 ... 1.82578742 1.78149609 1.7125984 ]
 [0.07135827 0.07135827 0.06397638 ... 1.87500001 1.81840552 1.76181104]
 [0.08858268 0.07874016 0.0738189  ... 1.89960632 1.83809056 1.78887797]]
The passing of i is  36


/tmp/ipykernel_50367/78431171.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_p['precip_local'] = current_p['precip_local'].fillna(0)
/tmp/ipykernel_50367/78431171.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_p['precip_local'] = valid_p['precip_local'].fillna(0)
/tmp/ipykernel_50367/78431171.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

36
36
in first loop /network/rit/lab/minderlab_rit/je845911/data/href/2022/20220217/nam_conusnest_2022021700f036.grib2
FINALFINALFINAL
[[0.05167323 0.03198819 0.03444882 ...        nan        nan        nan]
 [0.06151575 0.04675197 0.03690945 ...        nan        nan        nan]
 [0.06643701 0.06397638 0.0492126  ...        nan        nan        nan]
 ...
 [0.06397638 0.05905512 0.07135827 ... 1.82578742 1.78149609 1.7125984 ]
 [0.07135827 0.07135827 0.06397638 ... 1.87500001 1.81840552 1.76181104]
 [0.08858268 0.07874016 0.0738189  ... 1.89960632 1.83809056 1.78887797]]
FINALFINALFINAL


/network/rit/lab/snowclus/anaconda3_2021/envs/may21/lib/python3.8/site-packages/cartopy/crs.py:825: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/network/rit/lab/snowclus/anaconda3_2021/envs/may21/lib/python3.8/site-packages/cartopy/crs.py:877: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/network/rit/lab/snowclus/anaconda3_2021/envs/may21/lib/python3.8/site-packages/cartopy/crs.py:944: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(

England, John | 06/28/2022